# Import

## Modules

In [10]:
from pathlib import Path
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pymc3 as pm


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


 Setup path

In [2]:
PROJECT_ROOT = Path.cwd().parent.resolve()
sys.path.append(str(PROJECT_ROOT))

## Scripts

In [3]:
from src.data.make_dataset import get_cases_data, to_datetime, subset_latest_outbreak, get_daily_cases_stats

## Data

In [4]:
raw_cases_data = get_cases_data()

## Audit

In [5]:
raw_cases_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8964 entries, 0 to 8963
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   notification_date           8964 non-null   object 
 1   postcode                    8168 non-null   object 
 2   likely_source_of_infection  8964 non-null   object 
 3   lhd_2010_code               8157 non-null   object 
 4   lhd_2010_name               8157 non-null   object 
 5   lga_code19                  8157 non-null   float64
 6   lga_name19                  8157 non-null   object 
dtypes: float64(1), object(6)
memory usage: 490.3+ KB


In [6]:
raw_cases_data.head()

,notification_date,postcode,likely_source_of_infection,lhd_2010_code,lhd_2010_name,lga_code19,lga_name19
0,2020-01-25,2134,Overseas,X700,Sydney,11300.0,Burwood (A)
1,2020-01-25,2121,Overseas,X760,Northern Sydney,16260.0,Parramatta (C)
2,2020-01-25,2071,Overseas,X760,Northern Sydney,14500.0,Ku-ring-gai (A)
3,2020-01-27,2033,Overseas,X720,South Eastern Sydney,16550.0,Randwick (C)
4,2020-03-01,2077,Overseas,X760,Northern Sydney,14000.0,Hornsby (A)


# Preprocess data

In [7]:
raw_cases_data = to_datetime('notification_date', raw_cases_data)
data = subset_latest_outbreak('2021-06-01', 'Overseas', raw_cases_data)

In [8]:
data = get_daily_cases_stats(data)

In [9]:
data

,notification_date,Daily Number of Cases,Pct Change,Cumsum,Daily Difference,Growth Factor,Weekly Rolling Average,Epidemiological Days
0,2021-06-16,3,NaN,3,NaN,NaN,NaN,-9.0
1,2021-06-17,1,-0.666667,4,-2.0,NaN,NaN,-8.0
2,2021-06-18,2,1.000000,6,1.0,-0.500000,NaN,-7.0
3,2021-06-19,1,-0.500000,7,-1.0,-1.000000,NaN,-6.0
4,2021-06-20,2,1.000000,9,1.0,-1.000000,NaN,-5.0
5,2021-06-21,5,1.500000,14,3.0,3.000000,NaN,-4.0
6,2021-06-22,17,2.400000,31,12.0,4.000000,4.428571,-3.0
7,2021-06-23,12,-0.294118,43,-5.0,-0.416667,5.714286,-2.0
8,2021-06-24,21,0.750000,64,9.0,-1.800000,8.571429,-1.0
9,2021-06-25,28,0.333333,92,7.0,0.777778,12.285714,0.0


In [19]:
mask = data['Epidemiological Days'] == 0
initial_number_of_cases = data.loc[mask, 'Daily Number of Cases'].values[0]

mask = data['Epidemiological Days'] >= 0
daily_number_of_cases_std = data.loc[mask, 'Daily Number of Cases'].std()
average_growth_factor = data.loc[mask, 'Growth Factor'].mean()
std_growth_factor = data.loc[mask, 'Growth Factor'].std()


In [ ]:
# Create PyMC3 context manager
with pm.Model() as model:
    t = pm.Data("X", X)
    cases = pm.Data("y", y)

    # Intercept - We fixed this at 100.
    a = pm.Normal("a", mu=initial_number_of_cases, sigma=daily_number_of_cases_std)

    # Slope - Growth rate: 0.2 is approx value reported by others
    b = pm.Normal("b", mu=average_growth_factor, sigma=std_growth_factor)

    # Exponential regression
    growth = a * (1 + b) ** t

    # Likelihood error
    eps = pm.HalfNormal("eps")

    # Likelihood - Counts here, so poission or negative binomial. Causes issues. Lognormal tends to work better?
    pm.Lognormal("cases", mu=np.log(growth), sigma=eps, observed=cases)

    trace = pm.sample()
    post_Pred = pm.sample_posterior_predictive(trace)